### Simulation - Object Approach

- Background: I ran into two main problems when I tried to implement the simulation using just pandas (or when I tried to imitiate the code strucutre from the homework - ex: Patient Scheduling Problem)
    - 1) Patient Scheduling Problem was a 1-server model - whereas our initial problem statement has 6 unique terminals with different capacity (meaning 6 different servers) 
        - however can simplify this by assuming just one server, with infinite capacity
    - 2) Patient Scheduling Problem also didn't differentiate between the patient attributes. 
        - in the traffic problem however, each ship has its own 'periodicity' which should reset once each vessel arrives at the port.  
        - only when the periodicity is validated, can the ships begin to arrive at the port. 
        - so each ship is different from each other 
        - (Ships still arrive at a Poisson rate INTO the ports, but ships also follow their own distribution)
        
        
- Main Reason behind using objects: so that we can keep track of each ship + each terminal with its different attributes 
    - necessary attributes - ship: 
        - mean arrival time (round trip) 
        - cargo
        - length (only necessary if we are to constrain the terminals)
    - necesary attributes - terminal (if we are to implement) 
        - number of cranes
        - berth length 
        


References: [Using Simpy for simulation](https://medium.com/swlh/simulating-a-parallel-queueing-system-with-simpy-6b7fcb6b1ca1)

[2 server Queue Implementation - BhavyaJain](https://towardsdatascience.com/simulating-a-queuing-system-in-python-8a7d1151d485)
- ^[Github](https://github.com/BhavyaJain08/Bank-Simulation/blob/master/Bank_Simulation_Project.py)

- [Other Simpy Resources](https://pythonhosted.org/SimPy/Tutorials/TheBank.html#several-counters-with-individual-queues)

In [3]:
import numpy as np
import pandas as pd

### Vessel Information dataframe

In [8]:
def vessel_length(max_teu):
    ## SuperPX crane
    if max_teu > 14400:   
        v_length = 1300
        v_type = 2
    ## SuperPX crane
    elif max_teu > 10000:    
        v_length = 1200 
        v_type = 2
    ## Post PX crane
    elif max_teu > 5000:
        v_length = 1100
        v_type = 1
    ## Panamax Crane
    else: 
        v_length = 965
        v_type = 0
    return v_length, v_type

#### Vessel and Cargo Dataset

In [9]:
vessels = pd.read_csv("longbeach.csv")


## Apply the vessel type & length above and unpack the tuple to append to original dataframe
## Reference: https://stackoverflow.com/questions/56775952/unpacking-a-tuple-on-multiple-columns-of-a-dataframe-from-series-apply
vessels[['length','v_type']] = vessels['teu'].apply(lambda x: vessel_length(x)).tolist()
## Exclude vessles without any stops (did not cross the pacific)
vessels = vessels[vessels['stops']!=0].reset_index(drop=True)
vessels['mean_hr'] = vessels['mean'] * 24
vessels['stddev_hr'] = vessels['stddev'] * 24

## decision variable
## SEt 0 if able to be picked.....1 if not (still en route to port)
vessels['cycle_complete'] = 1


#### Terminal Capacity Dataset

In [10]:
## Will only look at Port of Long Beach + Terminals with Rail System
terminal_capacity = pd.read_csv("terminalcapacity.csv")
lb_port = terminal_capacity[(terminal_capacity['Port']=="Long Beach") & (terminal_capacity['Rail System']==1)]
# lb_port

In [11]:
orig = lb_port[['Name','Port','Berth','Yard','Rail System']]

## Take the floor of the multiplied output
## only reduce the crane #s + the container capacity 
reduced = np.floor(lb_port[['Crane_Panamax','Crane_PostPX','Crane_SuperPX','ContainerCap']].multiply(0.7, axis=1))
reduced_lb = pd.concat([orig, reduced], axis=1) 
reduced_lb['TotalCrane'] = reduced_lb[['Crane_Panamax','Crane_PostPX','Crane_SuperPX']].sum(axis=1)
reduced_lb['Avail_Berth'] = reduced_lb['Berth']   ## Necessary for later calculations
reduced_lb = reduced_lb.sort_values(by=['Name']).reset_index(drop=True)

### Class Implmentation

In [4]:
class Vessel:
    def __init__(self, period_mean, length, cargo, en_route): 
        self.period_mean = period_mean      # mean #of days to get to port 
        self.length = length                # vessel length
        self.cargo = cargo                  # Cargo Load   
        self.en_route = 0                   # 0 if still en route / 1 if able to be selected (initialize to 1)

    def __str__(self):
        return "This vessel comes to port every {} days and has a length of {} with current cargo of".format(self.period_mean, self.length)

In [6]:
class Terminal:
    def __init(self, berth_length, crane, at_capacity):
        self.berth_length = berth_length   # Length of the Berth at Terminal
        self.crane = crane                 # Number of Cranes at
        self.at_capacity = at_capacity     # 0 if berth does not have room left / 1 if it does (initialize to 1)
        
    def __str__(self):
        return "This vessel comes to port every {} days and has a length of {} with current cargo of".format(self.period_mean, self.length)
        

In [30]:
# for i in range(len(vessels)):
# #     vessel_i = Vessel(vessels.iloc[0]['mean'], vessels.iloc[0]['length'], 1)
#     holder = {i: Vessel(i=i, period_mean=vessels.iloc[i]['mean'], length=vessels.iloc[i]['length'], en_route=1)}
    

In [14]:
vessel_0 = Vessel(vessels.iloc[0]['mean'], vessels.iloc[0]['length'], vessels.iloc[0]['longbeach_cargo'], en_route = 1)
vessel_1 = Vessel(vessels.iloc[1]['mean'], vessels.iloc[1]['length'], vessels.iloc[1]['longbeach_cargo'], en_route = 1)
vessel_2 = Vessel(vessels.iloc[2]['mean'], vessels.iloc[2]['length'], vessels.iloc[2]['longbeach_cargo'], en_route = 1)
vessel_3 = Vessel(vessels.iloc[3]['mean'], vessels.iloc[3]['length'], vessels.iloc[3]['longbeach_cargo'], en_route = 1)
vessel_4 = Vessel(vessels.iloc[4]['mean'], vessels.iloc[4]['length'], vessels.iloc[4]['longbeach_cargo'], en_route = 1)
vessel_5 = Vessel(vessels.iloc[5]['mean'], vessels.iloc[5]['length'], vessels.iloc[5]['longbeach_cargo'], en_route = 1)
vessel_6 = Vessel(vessels.iloc[6]['mean'], vessels.iloc[6]['length'], vessels.iloc[6]['longbeach_cargo'], en_route = 1)
vessel_7 = Vessel(vessels.iloc[7]['mean'], vessels.iloc[7]['length'], vessels.iloc[7]['longbeach_cargo'], en_route = 1)
vessel_8 = Vessel(vessels.iloc[8]['mean'], vessels.iloc[8]['length'], vessels.iloc[8]['longbeach_cargo'], en_route = 1)
vessel_9 = Vessel(vessels.iloc[9]['mean'], vessels.iloc[9]['length'], vessels.iloc[9]['longbeach_cargo'], en_route = 1)
vessel_10 = Vessel(vessels.iloc[10]['mean'], vessels.iloc[10]['length'], vessels.iloc[10]['longbeach_cargo'], en_route = 1)
vessel_11 = Vessel(vessels.iloc[11]['mean'], vessels.iloc[11]['length'], vessels.iloc[11]['longbeach_cargo'], en_route = 1)
vessel_12 = Vessel(vessels.iloc[12]['mean'], vessels.iloc[12]['length'], vessels.iloc[12]['longbeach_cargo'], en_route = 1)
vessel_13 = Vessel(vessels.iloc[13]['mean'], vessels.iloc[13]['length'], vessels.iloc[13]['longbeach_cargo'], en_route = 1)
vessel_14 = Vessel(vessels.iloc[14]['mean'], vessels.iloc[14]['length'], vessels.iloc[14]['longbeach_cargo'], en_route = 1)
vessel_15 = Vessel(vessels.iloc[15]['mean'], vessels.iloc[15]['length'], vessels.iloc[15]['longbeach_cargo'], en_route = 1)
vessel_16 = Vessel(vessels.iloc[16]['mean'], vessels.iloc[16]['length'], vessels.iloc[16]['longbeach_cargo'], en_route = 1)
vessel_17 = Vessel(vessels.iloc[17]['mean'], vessels.iloc[17]['length'], vessels.iloc[17]['longbeach_cargo'], en_route = 1)
vessel_18 = Vessel(vessels.iloc[18]['mean'], vessels.iloc[18]['length'], vessels.iloc[18]['longbeach_cargo'], en_route = 1)
vessel_19 = Vessel(vessels.iloc[19]['mean'], vessels.iloc[19]['length'], vessels.iloc[19]['longbeach_cargo'], en_route = 1)
vessel_20 = Vessel(vessels.iloc[20]['mean'], vessels.iloc[20]['length'], vessels.iloc[20]['longbeach_cargo'], en_route = 1)
vessel_21 = Vessel(vessels.iloc[21]['mean'], vessels.iloc[21]['length'], vessels.iloc[21]['longbeach_cargo'], en_route = 1)
vessel_22 = Vessel(vessels.iloc[22]['mean'], vessels.iloc[22]['length'], vessels.iloc[22]['longbeach_cargo'], en_route = 1)
vessel_23 = Vessel(vessels.iloc[23]['mean'], vessels.iloc[23]['length'], vessels.iloc[23]['longbeach_cargo'], en_route = 1)
vessel_24 = Vessel(vessels.iloc[24]['mean'], vessels.iloc[24]['length'], vessels.iloc[24]['longbeach_cargo'], en_route = 1)
vessel_25 = Vessel(vessels.iloc[25]['mean'], vessels.iloc[25]['length'], vessels.iloc[25]['longbeach_cargo'], en_route = 1)
vessel_26 = Vessel(vessels.iloc[26]['mean'], vessels.iloc[26]['length'], vessels.iloc[26]['longbeach_cargo'], en_route = 1)

In [31]:

## This code template is from https://towardsdatascience.com/simulating-a-queuing-system-in-python-8a7d1151d485

class Traffic_Simulation:
    def __init__(self): 
        self.clock=0.0                      #simulation clock
        self.num_arrivals=0                 #total number of arrivals
        self.t_arrival=self.gen_int_arr()   #time of next arrival
        self.t_departure1=float('inf')      #departure time from server 1
        self.t_departure2=float('inf')      #departure time from server 2
        self.dep_sum1=0                     #Sum of service times by terminal 1
        self.dep_sum2=0                     #Sum of service times by terminal 2
        self.state_T1=0                     #current state of server1 (binary)
        self.state_T2=0                     #current state of server2 (binary)
        self.total_wait_time=0.0            #total wait time
        self.num_in_q=0                     #current number in queue
        self.number_in_queue=0              #vesels who had to wait in line(counter)
        self.num_in_system=0                #current number of vessels in system
        self.num_of_departures1=0           #number of customers served by terminal 1  
        self.num_of_departures2=0           #number of customers served by terminal 2 
        self.lost_customers=0               #customers who left without service
        
    def time_adv(self):                                                       #timing routine
        t_next_event=min(self.t_arrival, self.t_departure1, self.t_departure2)  #determine time of next event
        self.total_wait_time += (self.num_in_q * (t_next_event - self.clock))
        self.clock=t_next_event
                
        if self.t_arrival < self.t_departure1 and self.t_arrival < self.t_departure2:
            self.arrival()
        elif self.t_departure1 < self.t_arrival and self.t_departure1 < self.t_departure2:
            self.teller1()
        else:
            self.teller2()
            
    
    def arrival(self):              
        self.num_arrivals += 1
        self.num_in_system += 1

        if self.num_in_q == 0:                              #schedule next departure or arrival depending on state of servers
            if self.state_T1==1 and self.state_T2==1:
                self.num_in_q += 1
                self.number_in_queue+=1
                self.t_arrival=self.clock+self.gen_int_arr()
                
                
            elif self.state_T1==0 and self.state_T2==0:
                
                if np.random.choice([0,1])==1:
                    self.state_T1=1
                    self.dep1= self.gen_service_time_teller1()
                    self.dep_sum1 += self.dep1
                    self.t_departure1=self.clock + self.dep1
                    self.t_arrival=self.clock+self.gen_int_arr()

                else:
                    self.state_T2=1
                    self.dep2= self.gen_service_time_teller2()
                    self.dep_sum2 += self.dep2
                    self.t_departure2=self.clock + self.dep2
                    self.t_arrival=self.clock+self.gen_int_arr()

                    
            elif self.state_T1==0 and self.state_T2 ==1:    #if server 2 is busy customer goes to server 1
                self.dep1= self.gen_service_time_teller1()
                self.dep_sum1 += self.dep1
                self.t_departure1=self.clock + self.dep1
                self.t_arrival=self.clock+self.gen_int_arr()
                self.state_T1=1
            else:                                           #otherwise customer goes to server 2
                self.dep2= self.gen_service_time_teller2()
                self.dep_sum2 += self.dep2
                self.t_departure2=self.clock + self.dep2
                self.t_arrival=self.clock+self.gen_int_arr()
                self.state_T2=1
        
        elif self.num_in_q < 100 and self.num_in_q >= 1:
            self.num_in_q+=1
            self.number_in_queue+=1                             #if queue length is less than 4 generate next arrival and make customer join queue
            self.t_arrival=self.clock + self.gen_int_arr()

# Probability of losing customers due to long wait times 
        elif self.num_in_q == 4:                             #if queue length is 4 equal prob to leave or stay
            if np.random.choice([0,1])==0: 
                self.num_in_q+=1 
                self.number_in_queue+=1                 
                self.t_arrival=self.clock + self.gen_int_arr()
            else:
                self.lost_customers+=1
                
                
        elif self.num_in_q >= 5:                            #if queue length is more than 5 60% chance of leaving
            if np.random.choice([0,1],p=[0.4,0.6])==0:
                self.t_arrival=self.clock+self.gen_int_arr()
                self.num_in_q+=1 
                self.number_in_queue+=1 
            else:
                self.lost_customers+=1
                
        
    
    def teller1(self):              #departure from server 2
        self.num_of_departures1 += 1
        self.num_in_system -= 1 
        if self.num_in_q>0:
            self.dep1= self.gen_service_time_teller1()
            self.dep_sum1 += self.dep1
            self.t_departure1=self.clock + self.dep1
            self.num_in_q-=1
        else:
            self.t_departure1=float('inf') 
            self.state_T1=0                  
    
    def teller2(self):              #departure from server 1
        self.num_of_departures2 += 1
        self.num_in_system -= 1
        if self.num_in_q>0:
            self.dep2= self.gen_service_time_teller2()
            self.dep_sum2 += self.dep2
            self.t_departure2=self.clock + self.dep2
            self.num_in_q-=1
        else:
            self.t_departure2=float('inf')
            self.state_T2=0
            
    
    def gen_int_arr(self):                                             #function to generate arrival times using inverse trnasform
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 3)
    
    def gen_service_time_teller1(self):                                #function to generate service time for teller 1 using inverse trnasform
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 1.2)
    
    def gen_service_time_teller2(self):                                #function to generate service time for teller 1 using inverse trnasform
        return (-np.log(1-(np.random.uniform(low=0.0,high=1.0))) * 1.5)

s=Traffic_Simulation()
# df=pd.DataFrame(columns=['Average interarrival time','Average service time teller1','Average service time teller 2','Utilization teller 1','Utilization teller 2','People who had to wait in line','Total average wait time','Lost Customers'])
df=pd.DataFrame(columns=['Average interarrival time','Average service time teller1','Average service time teller 2','Utilization teller 1','Utilization teller 2','People who had to wait in line','Total average wait time','Lost Customers'])


for i in range(100):
    np.random.seed(i)
    s.__init__()
    while s.clock <= 240 :
        s.time_adv() 
    a=pd.Series([s.clock/s.num_arrivals,s.dep_sum1/s.num_of_departures1,s.dep_sum2/s.num_of_departures2,s.dep_sum1/s.clock,s.dep_sum2/s.clock,s.number_in_queue,s.total_wait_time,s.lost_customers],index=df.columns)
#         a=pd.Series([s.clock/s.num_arrivals, s.dep_sum1/s.num_of_departures1, s.dep_sum2/s.num_of_departures2, s.dep_sum1/s.clock, s.dep_sum2/s.clock, s.number_in_queue, s.total_wait_time],index=df.columns)
    df=df.append(a,ignore_index=True)   
    
df.to_excel('results.xlsx')  

### Practicing with Simpy

In [21]:
import simpy

In [22]:
def car(env):
     while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)

        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)

In [23]:
env = simpy.Environment()
env.process(car(env))

env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


In [24]:
class Car(object):
    def __init__ (self, env):
        self.env = env
        self.action = env.process(self.run())
        
    def run(self):
        while True:
            print("Start parking and charging at %d" % self.env.now)   ## current time (now)
            charge_duration = 5
            try: 
                yield self.env.process(self.charge(charge_duration))   ## Note that charge function is given below
            except simpy.Interrupt:
                print("Was interrupted. Hope the battery is full")
            
            print("Start driving at %d" % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
    
    def charge(self, duration):
        yield self.env.timeout(duration)   # "Timesout" for a given duration
        


In [25]:
    
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

In [26]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))

env.run(until=15)

Start parking and charging at 0
Was interrupted. Hope the battery is full
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


In [27]:
def car(env, name, bcs, driving_time, charge_duration):
    ## Driving to the BCS 
    yield env.timeout(driving_time)
    
    print("%s arriving at %d" % (name, env.now))
    with bcs.request() as req:
        yield req
        
        ## Charge Battery
        print("%s starting to charge at %s" % (name, env.now))
        yield env.timeout(charge_duration)
        print("%s leaving the bcs at %s" % (name, env.now))
        

In [28]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity = 3)
for i in range(4):
    env.process(car(env, 'Car %d' % i, bcs, i*2, 5))

In [29]:
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 2 starting to charge at 4
Car 0 leaving the bcs at 5
Car 3 arriving at 6
Car 3 starting to charge at 6
Car 1 leaving the bcs at 7
Car 2 leaving the bcs at 9
Car 3 leaving the bcs at 11
